In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from API.metadata import check_metadata_status


In [2]:
status = check_metadata_status()  # Replace with actual args if needed
print(status)


🔍 CHECKING METADATA STATUS...
Eötvös Loránd University Faculty of Law Library    |   4,359 |       0 |   0.0%
Auckland War Memorial Museum Tāmaki Paenga Hira    |     802 |       0 |   0.0%
Port Morien Digital Archive                        |   2,720 |       0 |   0.0%
Community Archives of Belleville & Hastings County |  17,221 |       0 |   0.0%
archivesatncbs                                     |      15 |       0 |   0.0%
Marshall Public Library                            |   2,939 |       0 |   0.0%
Boston Public Library Digital                      |     203 |       0 |   0.0%
San Francisco Public Library                       |     914 |       0 |   0.0%
CADL_LocalHistory                                  |   1,292 |       0 |   0.0%
Randolph Historical VT                             |     564 |       0 |   0.0%
LetterformArchive                                  |     657 |       0 |   0.0%
CommonsPrivacyTest                                 |       0 |       0 |   0.0%
CommonsTes

In [3]:
print(status)



[{'inst': {'nsid': '201268704@N06', 'date_launch': '1723824495', 'name': {'_content': 'CommonsPrivacyTest'}, 'urls': {'url': [{'type': 'site', '_content': 'https://www.flickr.com'}, {'type': 'license', '_content': 'http://www.getty.edu/legal/copyright.html#flickr'}, {'type': 'flickr', '_content': 'http://flickr.com/photos/201268704@N06/'}]}}, 'name': 'CommonsPrivacyTest', 'id': '201268704@N06', 'csv_filename': PosixPath('metadata/CommonsPrivacyTest.csv'), 'flickr_total': 0, 'csv_photos': 0, 'csv_rows': 1, 'coverage': 0}, {'inst': {'nsid': '126377022@N07', 'date_launch': '1409324867', 'name': {'_content': 'Internet Archive Book Images'}, 'urls': {'url': [{'type': 'site', '_content': 'http://archive.org'}, {'type': 'license', '_content': 'https://archive.org/about/terms.php'}, {'type': 'flickr', '_content': 'http://flickr.com/photos/internetarchivebookimages/'}]}}, 'name': 'Internet Archive Book Images', 'id': '126377022@N07', 'csv_filename': PosixPath('metadata/Internet Archive Book Ima

In [4]:
import plotly.express as px
import pandas as pd

# Filter out institutions with zero photos and create DataFrame
df = pd.DataFrame([item for item in status if item.get('flickr_total', 0) > 0])
df_treemap = df[['name', 'flickr_total']].rename(columns={'name': 'Institution', 'flickr_total': 'Photos'})

total_pictures = df['flickr_total'].sum()
print(f"Total number of pictures across all institutions: {total_pictures:,}")

# Create and display the treemap (opens in browser)
fig = px.treemap(
    df_treemap,
    path=['Institution'],  # Single level: each institution as a rectangle
    values='Photos',
    title='Flickr Commons Institutions by Photo Count',
    color='Institution',  # Color by institution name (random categorical colors)
    color_discrete_sequence=px.colors.qualitative.Set3,  # Distinct random colors per institution
    hover_data={
        'Institution': True,  # Institution name
        'Photos': ':,',       # Photo count
    }
)
fig.update_traces(
    textinfo='label+value+percent parent',
    hovertemplate='<b>%{label}</b><br>Photos: %{value:,}<br>%{percentParent:.1%}<extra></extra>'  # Custom hover: name, photos, percent
)
fig.show()

# Optional: Print top contributors for reference
top_insts = df.nlargest(10, 'flickr_total')[['name', 'flickr_total']]
print("Top 10 Institutions by Photo Count:")
print(top_insts.to_string(index=False))


Total number of pictures across all institutions: 843,218


Top 10 Institutions by Photo Count:
                                              name  flickr_total
             San Diego Air & Space Museum Archives        464541
                           The Library of Congress         44525
                                      NavyMedicine         24536
                          The National Archives UK         20156
                    Sinaloa Fotografías Históricas         19241
Community Archives of Belleville & Hastings County         17221
                            U.S. National Archives         16931
                     Mennonite Church USA Archives         13464
                    Colección Amadeo León - Boconó         12557
                 SMU Libraries Digital Collections         11399


In [ ]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import json

# Setup directory
metadata_dir = Path("../metadata")
metadata_dir.mkdir(exist_ok=True)

# Load all CSVs (assumes they have the specified columns)
csv_files = list(metadata_dir.glob("*.csv"))
if not csv_files:
    print("No CSVs found. Generating sample data for demo (remove this block for real data).")
    # Sample data generation (comment out for real use)
    sample_data = {
        'id': range(1, 101),
        'secret': [f'sec{i}' for i in range(1, 101)],
        'title': [f'Sample Title {i}' for i in range(1, 101)],
        'description': ['This is a sample description with varying lengths. ' * (i % 10 + 1) for i in range(1, 101)],
        'date_taken': pd.date_range('2020-01-01', periods=100, freq='D'),
        'date_uploaded': pd.date_range('2020-01-02', periods=100, freq='D'),
        'latitude': np.random.normal(46.5, 0.5, 100).clip(40, 50),
        'longitude': np.random.normal(6.6, 0.5, 100).clip(2, 10),
        'comments': [f'comment {i}' for i in range(1, 101)],
        'size': np.random.randint(1000, 10000, 100),
        'image_url': [f'http://example.com/img{i}.jpg' for i in range(1, 101)],
        'notes': [f'note {i}' for i in range(1, 101)],
        'tags': ['tag1, tag2'] * 100
    }
    files = ['file1.csv', 'file2.csv', 'file3.csv']
    for idx, fname in enumerate(files):
        df = pd.DataFrame(sample_data)
        df['description'] = df['description'].apply(lambda x: x * (idx + 1))
        df['date_taken'] = df['date_taken'] + pd.Timedelta(days=idx * 30)
        df['latitude'] += idx * 0.1
        df['longitude'] += idx * 0.2
        df.to_csv(metadata_dir / fname, index=False)
    csv_files = list(metadata_dir.glob("*.csv"))

print(f"Processing {len(csv_files)} files: {[f.name for f in csv_files]}")

# Combine data with file labels AND row indices
all_data = []
for f in csv_files:
    df = pd.read_csv(f)
    df['file'] = f.name
    df['row_index'] = range(len(df))  # Add 0-based row index for each CSV
    all_data.append(df)
combined_df = pd.concat(all_data, ignore_index=True)

# Compute lengths
combined_df['desc_len'] = combined_df['description'].str.len()
combined_df['title_len'] = combined_df['title'].str.len()

# Custom hover template showing: CSV, Row, ID, Title, Value
hover_template = (
    "<b>CSV: %{customdata[0]}</b><br>" +
    "Row: %{customdata[1]}<br>" +
    "ID: %{customdata[2]}<br>" +
    "<b>Title: %{customdata[3]}</b><br>" +
    "Value: %{y:.0f}<extra></extra>"

)

# 1. Dates distribution (histogram)
fig_dates = px.histogram(combined_df, x='date_taken', color='file',
                         title='date_taken Distribution by CSV File',
                         labels={'date_taken': 'Date Taken', 'count': 'Frequency'})
fig_dates.update_traces(
    customdata=combined_df[['file', 'row_index', 'id', 'title']].values,
    hovertemplate=hover_template
)
fig_dates.show()

# 2. Description length distribution
fig_desc = px.histogram(combined_df, x='desc_len', color='file', nbins=50,
                        title='Description Length (chars) Distribution by CSV',
                        labels={'desc_len': 'Length (chars)', 'count': 'Frequency'})
fig_desc.update_traces(
    customdata=combined_df[['file', 'row_index', 'id', 'title']].values,
    hovertemplate=hover_template
)
fig_desc.show()

# 3. Title length distribution
fig_title = px.histogram(combined_df, x='title_len', color='file', nbins=30,
                         title='Title Length (chars) Distribution by CSV',
                         labels={'title_len': 'Length (chars)', 'count': 'Frequency'})
fig_title.update_traces(
    customdata=combined_df[['file', 'row_index', 'id', 'title']].values,
    hovertemplate=hover_template
)
fig_title.show()

# 4. Lat/Lon 2D scatter (special hover for 2D)
hover_map_template = (
    "<b>CSV: %{customdata[0]}</b><br>" +
    "Row: %{customdata[1]}<br>" +
    "ID: %{customdata[2]}<br>" +
    "Title: %{customdata[3]}<br>" +
    "Lat: %{y:.4f}<br>" +
    "Lon: %{x:.4f}<extra></extra>"
)

fig_map = px.scatter(combined_df, x='longitude', y='latitude', color='file',
                     title='Latitude vs Longitude Distribution by CSV',
                     labels={'latitude': 'Latitude', 'longitude': 'Longitude'})
fig_map.update_traces(
    customdata=combined_df[['file', 'row_index', 'id', 'title']].values,
    hovertemplate=hover_map_template
)
fig_map.show()

Processing 3 files: ['file2.csv', 'file1.csv', 'file3.csv']


In [ ]:
# 2. Description length distribution
fig_desc = px.histogram(combined_df, x='desc_len', color='file', nbins=50,
                        title='Description Length (chars) Distribution by CSV',
                        labels={'desc_len': 'Length (chars)', 'count': 'Frequency'})
fig_desc.show()
# fig_desc.write_image("desc_length_distribution.png")


In [ ]:

# 3. Title length distribution
fig_title = px.histogram(combined_df, x='title_len', color='file', nbins=30,
                         title='Title Length (chars) Distribution by CSV',
                         labels={'title_len': 'Length (chars)', 'count': 'Frequency'})
fig_title.show()
# fig_title.write_image("title_length_distribution.png")



In [ ]:
# 4. Lat/Lon 2D scatter → MAPBOX MAP with world outline + 1:1 ratio
fig_map = px.scatter_mapbox(
    combined_df, 
    lat='latitude', 
    lon='longitude', 
    color='file',
    hover_name='title',  # Title as main hover label
    custom_data=['file', 'row_index', 'id'],  # Extra hover data
    zoom=2,  # Adjust zoom (0=world, 10=city)
    height=600,
    mapbox_style="open-street-map",  # Free world map tiles
    title='Geographic Distribution by CSV (Mapbox)'
)

# Enhanced hover showing ALL requested info
fig_map.update_traces(
    hovertemplate=(
        "<b>CSV: %{customdata[0]}</b><br>" +
        "Row: %{customdata[1]}<br>" +
        "ID: %{customdata[2]}<br>" +
        "<b>Title: %{hover_name}</b><br>" +
        "Lat: %{lat:.4f}<br>" +
        "Lon: %{lon:.4f}<extra></extra>"
    )
)

# Lock 1:1 aspect ratio + clean layout
fig_map.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=46.5, lon=6.6),  # Lausanne-centered
        zoom=4
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, t=50, b=0)
)

# Alternative: equirectangular projection for true 1:1 degrees
# fig_map.update_geos(projection_type="equirectangular", 
#                     lataxis_range=[45,48], lonaxis_range=[5,8],
#                     constraintratio=True)

fig_map.show()

NameError: name 'px' is not defined